In [1]:
import requests
from dotenv import load_dotenv
import pandas as pd
import os
pd.options.display.max_columns = 999


load_dotenv()


# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(os.environ.get("NAME"), os.environ.get("SECRET"))

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': os.environ.get("USERNAME"),
        'password': os.environ.get("PASSCOMPTE")}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}



## Request post

In [2]:
data = requests.get('https://oauth.reddit.com/r/AITAH/hot', headers=headers, params={'limit':'50'}).json()["data"]["children"]
data = pd.json_normalize(data)
data.columns = [c.replace("data.", "") for c in data.columns]  

In [3]:
posts = data[["title", "selftext", "link_flair_text", "score", "created", "edited", "author", "permalink"]]

In [4]:
posts.head()

,title,selftext,link_flair_text,score,created,edited,author,permalink
0,r/AITAH Lounge,A place for members of r/AITAH to chat with ea...,None,903,1.615940e+09,False,DepressedTrashKitty,/r/AITAH/comments/m6n71o/raitah_lounge/
1,AITAH for breaking up with my girlfriend when ...,When I was 16 years old my girlfriend broke up...,None,10432,1.718833e+09,False,AdAlarmed2847,/r/AITAH/comments/1djuriw/aitah_for_breaking_u...
2,AITA for kicking my husband out of the house a...,"I (28F) have been married to my husband, Steve...",Advice Needed,1611,1.718851e+09,False,hardlessonsThroRa,/r/AITAH/comments/1dk0wi5/aita_for_kicking_my_...
3,AITAH for approaching women at the bar while m...,My wife (35f) and I (36m) went to a bar last w...,None,6548,1.718813e+09,False,Many_Major2827,/r/AITAH/comments/1djmo6t/aitah_for_approachin...
4,AITAH for not giving my wife the full reason t...,My wife and I have been discussing baby names ...,None,10049,1.718806e+09,1718830939.0,Bitter-Marketing-623,/r/AITAH/comments/1djk6v8/aitah_for_not_giving...


## Classify posts

In [28]:
import torch
torch.cuda.is_available()

True

['neutral', 'sadness', 'surprise']

In [41]:
from transformers import pipeline
from tqdm import tqdm
import numpy as np

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

labels = []
for title in tqdm(posts.title):
    
    labels_post = classifier(title)
    list_label_post = []
    for detail_label in labels_post[0]:
        if detail_label["score"] > 0.20: ## TODO: Threshold not analysied
            list_label_post.append(detail_label["label"])
    labels.append(list_label_post)

labels = pd.Series(labels, name="label")

/home/romain/anaconda3/envs/projet/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/romain/anaconda3/envs/projet/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
100%|██████████| 51/51 [00:02<00:00, 24.41it/s]


In [43]:
classified_posts = pd.concat((posts, labels), axis=1)
classified_posts[["title", "selftext", "label"]]

,title,selftext,label
0,r/AITAH Lounge,A place for members of r/AITAH to chat with ea...,[neutral]
1,AITAH for breaking up with my girlfriend when ...,When I was 16 years old my girlfriend broke up...,[fear]
2,AITA for kicking my husband out of the house a...,"I (28F) have been married to my husband, Steve...",[disgust]
3,AITAH for approaching women at the bar while m...,My wife (35f) and I (36m) went to a bar last w...,[disgust]
4,AITAH for not giving my wife the full reason t...,My wife and I have been discussing baby names ...,"[anger, disgust]"
5,Family forgot I was a Father.,"\nSo, literally, both kids and my wife, all wh...","[neutral, sadness, surprise]"
6,AITAH for refusing to change items on my dinne...,I will try to keep this short. We are having a...,"[anger, neutral, surprise]"
7,AITA for putting in a claim to my late father’...,[Original post](https://www.reddit.com/r/AITAH...,"[anger, neutral, sadness]"
8,AITAH for not wanting a baby girl so my MIL wo...,I 28(F) am pregnant and about to deliver our f...,"[sadness, surprise]"
9,AITAH for not spending Father’s Day with my wi...,I (34M) have been married to my wife (32F) for...,"[anger, disgust, surprise]"


In [50]:
from pprint import pprint
pprint(classified_posts.iloc[36].title)

('AITAH for telling my boyfriend if he doesn’t want to have sex with me, then '
 'we need to break up? ')


## Request comments

In [66]:
comments = requests.get('https://oauth.reddit.com'+posts.permalink[0], headers=headers, params={'limit':'50'}).json()

In [26]:
# comments